In [ ]:
rm -rf /kaggle/working/*

In [ ]:
!git clone --branch main https://github.com/giankev/Ancient-to-Modern-Italian-Automatic-Translation.git

In [ ]:
from sklearn.metrics import accuracy_score, cohen_kappa_score
from scipy.stats import pearsonr, spearmanr

def metric_score(path_human_score, path_LLM_judge_score):

    human_score = pd.read_csv(path_human_score)
    LLM_judge_score = pd.read_csv(path_LLM_judge_score)
    
    LLM_judge_score = LLM_judge_score["score"]
    human_score = human_score["human score"]
    
    kappa = cohen_kappa_score(human_score, LLM_judge_score)
    spearman, _ = spearmanr(human_score, LLM_judge_score)

    return kappa, spearman

In [ ]:
experiments = [
    ("gemma2b_it_context_learning",
     "gemini-2.0-flash-lite",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/human_score/evaluation_human_judge_gemma2b-it_context_learning.csv",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/judge_gemma-2b-it_context_learning/CulturalIA-hw2_transl-judge.csv"
    ),

    ("gemma2b_it_context_learning",
     "prometheus",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/human_score/evaluation_human_judge_gemma2b-it_context_learning.csv",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/prometheus/Gemma2b-it-evaluations_prometheus.csv"
    ),

    ("opusMT",
     "gemini-2.0-flash-lite",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/human_score/evaluation_human_judge_opusMT.csv",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/judge_opusMT/CulturalIA-hw2_transl-judge.csv"   
    ),

    ("opusMT",
     "prometheus",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/human_score/evaluation_human_judge_opusMT.csv",
     "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/prometheus/OpusMT-it-evaluations_prometheus.csv"   
    ) 
]

results = []

for model_name,llm_judge, path_human, path_llm in experiments:
    kappa, spearman = metric_score(path_human, path_llm)
    results.append({
        "model": model_name,
        "llm judge": llm_judge,
        "cohen_kappa": round(kappa, 4),
        "spearman_correlation": round(spearman, 4)
    })

    print(f"Aggiunto: model={model_name}, judge={llm_judge}, kappa={round(kappa, 4)}, spearman={round(spearman, 4)}")

results_df = pd.DataFrame(results)
results_df.to_csv("metric_score.csv", index=False, sep=";", decimal = ",")

print("scores saved")